In [ ]:
# Cell 1: Imports and Data Loading
import sys
sys.path.append('../src')
from portfolio import *
import pandas as pd

daily_returns = pd.read_csv('../data/processed/daily_returns.csv', index_col=0, parse_dates=True)
tsla_future_forecast = pd.read_csv('../data/processed/tsla_future_forecast.csv', index_col=0, parse_dates=True, header=None, names=['date', 'forecast_price'])

# Calculate daily returns for the TSLA forecast
tsla_forecast_returns = tsla_future_forecast['forecast_price'].pct_change().dropna()

# Cell 2: Portfolio Optimization
ef, max_sharpe_weights, max_sharpe_stats, min_vol_weights, min_vol_stats = optimize_portfolio(daily_returns, tsla_forecast_returns)

# Cell 3: Plot the Efficient Frontier
plot_efficient_frontier(ef)

# Cell 4: Final Recommendation
print("\n--- Recommended Portfolio (Maximum Sharpe Ratio) ---")
print("Optimal Weights:")
for ticker, weight in max_sharpe_weights.items():
    print(f"\t{ticker}: {weight:.2%}")
print("\nPerformance Metrics (Annualized):")
print(f"\tExpected Return: {max_sharpe_stats[0]*100:.2f}%")
print(f"\tVolatility: {max_sharpe_stats[1]*100:.2f}%")
print(f"\tSharpe Ratio: {max_sharpe_stats[2]:.2f}")

# You can choose max_sharpe or min_vol based on your analysis.
# Save the chosen portfolio weights for backtesting
pd.Series(max_sharpe_weights).to_csv('../data/processed/optimal_weights.csv', header=True)

In [ ]:
# Cell 5: Backtesting Setup
import sys
sys.path.append('../src')
from backtest import *
import pandas as pd

daily_returns = pd.read_csv('../data/processed/daily_returns.csv', index_col=0, parse_dates=True)
optimal_weights = pd.read_csv('../data/processed/optimal_weights.csv', index_col=0, header=None, squeeze=True).to_dict()

backtest_start = '2024-08-01'
backtest_end = '2025-07-31'

# Cell 6: Simulate Strategy and Benchmark
# Your Strategy
strategy_cumulative_returns = backtest_portfolio(daily_returns, optimal_weights, backtest_start, backtest_end)

# Benchmark (60% SPY, 40% BND)
benchmark_weights = {'SPY': 0.6, 'BND': 0.4}
benchmark_cumulative_returns = backtest_portfolio(daily_returns, benchmark_weights, backtest_start, backtest_end)

# Cell 7: Performance Analysis
plot_backtest_results(strategy_cumulative_returns, benchmark_cumulative_returns)

strategy_returns_series = (1 + daily_returns.loc[backtest_start:backtest_end] * pd.Series(optimal_weights)).sum(axis=1).pct_change().dropna()
benchmark_returns_series = (1 + daily_returns.loc[backtest_start:backtest_end] * pd.Series(benchmark_weights)).sum(axis=1).pct_change().dropna()

strategy_sharpe = calculate_sharpe_ratio(strategy_returns_series)
benchmark_sharpe = calculate_sharpe_ratio(benchmark_returns_series)

print("\n--- Backtest Summary ---")
print(f"Strategy Total Return: {strategy_cumulative_returns.iloc[-1] - 1:.2%}")
print(f"Benchmark Total Return: {benchmark_cumulative_returns.iloc[-1] - 1:.2%}")
print(f"Strategy Annualized Sharpe Ratio: {strategy_sharpe:.2f}")
print(f"Benchmark Annualized Sharpe Ratio: {benchmark_sharpe:.2f}")

# Final conclusion
if strategy_sharpe > benchmark_sharpe:
    print("\nConclusion: The optimized strategy outperformed the benchmark on a risk-adjusted basis.")
else:
    print("\nConclusion: The benchmark performed better than the optimized strategy.")